Attention
===

데이터 출처 :https://www.data.go.kr/data/15051872/fileData.do(참고, 데이터 수정)   
데이터 설명 : 서울특별시_공공자전거 시간대별 이용정보   
data url = https://raw.githubusercontent.com/Datamanim/datarepo/main/bicycle/seoul_bi.csv

In [15]:
import pandas as pd

dataurl = 'https://raw.githubusercontent.com/Datamanim/datarepo/main/bicycle/seoul_bi.csv'
df = pd.read_csv(dataurl)
df.head()

,대여일자,대여시간,대여소번호,대여구분코드,성별,연령대코드,이용건수,운동량,탄소량,이동거리,사용시간
0,2021-06-01,0,3541,정기권,F,~10대,1,0.00,0.00,0.00,8
1,2021-06-01,0,765,정기권,F,~10대,1,27.21,0.35,1526.81,19
2,2021-06-01,0,2637,정기권,F,~10대,1,41.40,0.37,1608.56,18
3,2021-06-01,0,2919,정기권,F,~10대,1,0.00,0.00,0.00,75
4,2021-06-01,0,549,정기권,F,~10대,1,13.04,0.17,731.55,6


**Question 31**

대여 일자별 데이터의 수를 데이터프레임으로 출력하고, 가장 많은 데이터가 있는 날짜를 출력하라.

In [28]:
result = df['대여일자'].value_counts().sort_index().to_frame()
Ans = result[result['count'] == result['count'].max()].index[0]

display(result)
Ans

,count
대여일자,
2021-06-01,48081
2021-06-02,56100
2021-06-03,18985
2021-06-04,56788
2021-06-05,52730
2021-06-06,47787
2021-06-07,52623
2021-06-08,55623
2021-06-09,54507


'2021-06-04'

**Question 32**

각 일자의 요일을 표기하고 (‘Monday’ ~’Sunday’) ‘day_name’ 컬럼을 추가하고 이를 이용하여 각 요일별 이용 횟수의 총합을 데이터 프레임으로 출력하라.

In [38]:
df['day_name'] = pd.to_datetime(df['대여일자']).dt.day_name()
Ans = df['day_name'].value_counts().to_frame()
Ans

,count
day_name,
Wednesday,110607
Friday,108877
Monday,107568
Tuesday,103704
Saturday,101299
Sunday,90918
Thursday,55977


**Question 33**

각 요일별 가장 많이 이용한 대여소의 이용횟수와 대여소 번호를 데이터 프레임으로 출력하라.

In [51]:
result = df.groupby(['day_name', '대여소번호']).size().to_frame('size').sort_values(['day_name', 'size'], ascending=False).reset_index()
Ans = result.drop_duplicates('day_name', keep='first').reset_index(drop=True)
display(Ans)

,day_name,대여소번호,size
0,Wednesday,502,282
1,Tuesday,502,267
2,Thursday,2715,137
3,Sunday,502,372
4,Saturday,502,378
5,Monday,502,242
6,Friday,502,277


In [48]:
# My Answer
Ans = df.groupby(['day_name', '대여소번호'], as_index=False).size().sort_values(['day_name', 'size'], ascending=False).drop_duplicates('day_name').reset_index(drop=True)
Ans

,day_name,대여소번호,size
0,Wednesday,502,282
1,Tuesday,502,267
2,Thursday,2715,137
3,Sunday,502,372
4,Saturday,502,378
5,Monday,502,242
6,Friday,502,277


**Question 34**

나이대별 대여구분 코드의 (일일권/전체횟수) 비율을 구한 후 가장 높은 비율을 가지는 나이대를 확인하라. 일일권의 경우 일일권과 일일권(비회원)을 모두 포함하라.

In [60]:
daily = df[df.대여구분코드.str.contains('일일권')].연령대코드.value_counts().sort_index()
total = df.연령대코드.value_counts().sort_index()

ratio = (daily / total).sort_values(ascending=False)
print(ratio)
print('max ratio age ', ratio.index[0])

연령대코드
~10대    0.312525
20대     0.239913
30대     0.166702
40대     0.110585
50대     0.065414
70대~    0.048962
60대     0.042462
Name: count, dtype: float64
max ratio age  ~10대


**Question 35**

연령대별 평균 이동거리를 구하여라.

In [63]:
Ans = df[['연령대코드', '이동거리']].groupby('연령대코드').mean()
Ans

,이동거리
연령대코드,
20대,3211.890552
30대,3341.443859
40대,3514.857416
50대,3593.668100
60대,3538.145737
70대~,3085.039641
~10대,2634.426279


**Question 36**

연령대 코드가 20대인 데이터를 추출하고,이동거리값이 추출한 데이터의 이동거리값의 평균 이상인 데이터를 추출한다.최종 추출된 데이터를 대여일자, 대여소 번호 순서로 내림차순 정렬 후 1행부터 200행까지의 탄소량의 평균을 소숫점 3째 자리까지 구하여라.

In [79]:
tw = df[df.연령대코드 =='20대'].reset_index(drop=True)
tw_mean = tw[tw.이동거리 >= tw.이동거리.mean()].reset_index(drop=True)
tw_mean['탄소량'] =tw_mean['탄소량'].astype('float')
target =tw_mean.sort_values(['대여일자','대여소번호'], ascending=False).reset_index(drop=True).iloc[:200].탄소량
result = round(target.sum()/len(target),3)
print(result)

1.613


In [78]:
# My Answer
target = df[df['연령대코드']=='20대']
target2 = target[target['이동거리'] >= target['이동거리'].mean()]
co2_amount = target2.sort_values(['대여일자', '대여소번호'], ascending=False).reset_index(drop=True).iloc[:200, 8].astype('float').mean()
round(co2_amount, 3)

1.613

**Question 37**

6월 7일 ~10대의 “이용건수”의 중앙값은?